In [2]:
from bs4 import BeautifulSoup
import numpy
import requests as res
from urllib.parse import urlparse, parse_qs, urljoin
import json
import openpyxl
import time
import random
from selenium import webdriver
import re
import pandas as pd

# 연단위로 페이지 긁어오기

In [3]:
def get_player_year_link():
    chromedriver="./chromedriver"
    driver=webdriver.Chrome(chromedriver)
    basic_url="https://www.fifaindex.com/players/"
    driver.get(basic_url)
    driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[2]/ol/li[2]/a').click()
    driver.implicitly_wait(2)
    html=driver.page_source
    driver.quit()
    soup=BeautifulSoup(html,"lxml",from_encoding="utf-8")
    content=soup.select("ol.breadcrumb > li > ul")[0]
    fifa_year_links = [{"title": a_tag.text, "link": urljoin(basic_url, a_tag["href"])}
                          for a_tag in content.find_all("a")]
    
    return fifa_year_links

In [6]:
a=get_player_year_link()

c:\users\rhlfu\anaconda3\lib\site-packages\bs4\__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [66]:
a

[{'link': 'https://www.fifaindex.com/players/', 'title': 'FIFA 18'},
 {'link': 'https://www.fifaindex.com/players/fifa17_173/', 'title': 'FIFA 17'},
 {'link': 'https://www.fifaindex.com/players/fifa16_73/', 'title': 'FIFA 16'},
 {'link': 'https://www.fifaindex.com/players/fifa15_14/', 'title': 'FIFA 15'},
 {'link': 'https://www.fifaindex.com/players/fifa14_13/', 'title': 'FIFA 14'},
 {'link': 'https://www.fifaindex.com/players/fifa13_10/', 'title': 'FIFA 13'},
 {'link': 'https://www.fifaindex.com/players/fifa12_9/', 'title': 'FIFA 12'},
 {'link': 'https://www.fifaindex.com/players/fifa11_7/', 'title': 'FIFA 11'},
 {'link': 'https://www.fifaindex.com/players/fifa10_6/', 'title': 'FIFA 10'},
 {'link': 'https://www.fifaindex.com/players/fifa09_5/', 'title': 'FIFA 09'},
 {'link': 'https://www.fifaindex.com/players/fifa08_4/', 'title': 'FIFA 08'},
 {'link': 'https://www.fifaindex.com/players/fifa07_3/', 'title': 'FIFA 07'},
 {'link': 'https://www.fifaindex.com/players/fifa06_2/', 'title': '

# 각 연도별 선수url 긁어오기

In [7]:
def merge_two_dicts(x, y):
    """Given two dicts, merge them into a new dict as a shallow copy."""
    for c,t in zip(x,y):
        c.update(t)
    return x

In [9]:
def year_page_cralwer(fifa_year_link,start_page,end_page,que):
    target_player=[]  # url을 담을 리스트
    year_only_link=fifa_year_link["link"]
    is_unlast=True
    basic_url="https://www.fifaindex.com/players/"
    while is_unlast:
            link=year_only_link+str(start_page)+"/"
            get_page=res.get(link)
            page=BeautifulSoup(get_page.content,"lxml")
            player_url=page.select("tbody > tr > td:nth-of-type(4)") #url
            club=page.find_all("",{"class":"team small"})# 선수 클럽
            nation=page.find_all("",{"class":"nation small"}) #선수 나라
            
            
            player=[{"player_name": player.find("a").get_text(),"url":urljoin(basic_url,player.find("a")["href"])} for player in player_url]
            # 선수 이름과 url을 조합
            team=[{"team":list(i)[1]["title"],"nation":list(i)[0]["title"]}for i in list(zip(nation,club))]
            #선수 클럽과 나라를 조합
            player_team=merge_two_dicts(player,team)
            #선수 모든 데이터 규합
#             if player_team not in target_player:# 규합된 개인의 선수 데이터를 하나씩 쌓음
#                 target_player+=player_team
            
    
            que.put(player_team)  # 선수를 큐에 쌓는다 
            
            
            if start_page==end_page: #페이지가 마지막 페이지면 나간다.
                break
            start_page+=1
            time.sleep(0.1)
        
#     return target_player
            
                

In [10]:
import queue
from threading import Thread

In [18]:
for j in range(4):
    print(j*150+1)
    print(j*150+150)

1
150
151
300
301
450
451
600


In [13]:
que=queue.Queue(maxsize=0)


thread_list=[]
for i in range(4):
    worker=Thread(target=year_page_cralwer,args=(a[0],i*150+1,150*i+150,que))
    worker.setDaemon(True)
    thread_list.append(worker)
    worker.start()

for k in thread_list:
    k.join()
    
    
    


# 풀링으로 맥스 프로세스로 작업해보기


In [16]:
while not que.empty():
    print(que.get())
    print("\n")

In [62]:
a[0]["link"]+str(1)+"/"

'https://www.fifaindex.com/players/1/'

In [136]:
basic_url="https://www.fifaindex.com/players/"
get_page=res.get(a[0]["link"]+str(1)+"/")
page=BeautifulSoup(get_page.content,"lxml")
a_tags=page.select("tbody > tr > td:nth-of-type(4)")
club=page.find_all("",{"class":"team small"})
nation=page.find_all("",{"class":"nation small"})



k=[{"player_name": player.find("a").get_text(),
    "url":urljoin(basic_url,player.find("a")["href"])} for player in a_tags]
t=[{"team":list(i)[0]["title"],"nation":list(i)[1]["title"]}for i in list(zip(nation,club))]   
b=merge_two_dicts(k,t)

In [15]:
basic_url="https://www.fifaindex.com/players/2/"
get_page=res.get(a[0]["link"]+str(2)+"/")
page=BeautifulSoup(get_page.content,"lxml")
a_tags=page.select("tbody > tr > td:nth-of-type(4)")
club=page.find_all("",{"class":"team small"})
nation=page.find_all("",{"class":"nation small"})



k=[{"player_name": player.find("a").get_text(),
    "url":urljoin(basic_url,player.find("a")["href"])} for player in a_tags]
t=[{"team":list(i)[0]["title"],"nation":list(i)[1]["title"]}for i in list(zip(nation,club))]   
n=merge_two_dicts(k,t)

In [12]:
b=year_page_cralwer(a[0],1,2)

In [13]:
c=pd.DataFrame(b)

In [14]:
c

,nation,player_name,team,url
0,Portugal,Cristiano Ronaldo,Real Madrid,https://www.fifaindex.com/player/20801/cristia...
1,Argentina,Lionel Messi,FC Barcelona,https://www.fifaindex.com/player/158023/lionel...
2,Brazil,Neymar,PSG,https://www.fifaindex.com/player/190871/neymar/
3,Uruguay,Luis Suárez,FC Barcelona,https://www.fifaindex.com/player/176580/luis-s...
4,Germany,Manuel Neuer,FC Bayern,https://www.fifaindex.com/player/167495/manuel...
5,Poland,Robert Lewandowski,FC Bayern,https://www.fifaindex.com/player/188545/robert...
6,Spain,De Gea,Manchester Utd,https://www.fifaindex.com/player/193080/de-gea/
7,Belgium,Kevin De Bruyne,Manchester City,https://www.fifaindex.com/player/192985/kevin-...
8,Belgium,Eden Hazard,Chelsea,https://www.fifaindex.com/player/183277/eden-h...
9,Germany,Toni Kroos,Real Madrid,https://www.fifaindex.com/player/182521/toni-k...


In [29]:
pag=res.get("https://www.fifaindex.com/players/fifa17_173/2/")
pag=BeautifulSoup(pag.content,"lxml")
a_tag=pag.select("t table_striped players td a")


In [54]:
player_tag=pag.select("tbody > tr > td:nth-of-type(4)") # td에서 네번째 데이버만 긁어와라
for a in player_tag:
    url=a.find("a")["href"]
    name=a.find("a").get_text()
    print(a)

<td data-title="Name"><a href="/player/1179/gianluigi-buffon/fifa17_173/" title="Gianluigi Buffon">Gianluigi Buffon</a></td>
<td data-title="Name"><a href="/player/41/iniesta/fifa17_173/" title="Iniesta">Iniesta</a></td>
<td data-title="Name"><a href="/player/200389/jan-oblak/fifa17_173/" title="Jan Oblak">Jan Oblak</a></td>
<td data-title="Name"><a href="/player/198710/james-rodr%C3%ADguez/fifa17_173/" title="James Rodríguez">James Rodríguez</a></td>
<td data-title="Name"><a href="/player/188567/pierre-emerick-aubameyang/fifa17_173/" title="Pierre-Emerick Aubameyang">Pierre-Emerick Aubameyang</a></td>
<td data-title="Name"><a href="/player/184344/leonardo-bonucci/fifa17_173/" title="Leonardo Bonucci">Leonardo Bonucci</a></td>
<td data-title="Name"><a href="/player/181872/arturo-vidal/fifa17_173/" title="Arturo Vidal">Arturo Vidal</a></td>
<td data-title="Name"><a href="/player/168651/ivan-rakiti%C4%87/fifa17_173/" title="Ivan Rakitić">Ivan Rakitić</a></td>
<td data-title="Name"><a hre

In [34]:
nation=pag.find_all("",{"class":"nation small"})
for a in nation:
    print (a["title"])

Italy
Spain
Slovenia
Colombia
Gabon
Italy
Chile
Croatia
Spain
Slovenia
Spain
Netherlands
Argentina
Italy
Austria
Armenia
Germany
Denmark
Germany
Spain
Spain
Spain
Brazil
Argentina
Spain
Uruguay
Belgium
Brazil
Italy
Slovakia


In [35]:
team=pag.find_all("",{"class":"team small"})
for a in team:
    print (a["title"])

Juventus
FC Barcelona
Atlético Madrid
Real Madrid
Bor. Dortmund
Juventus
FC Bayern
FC Barcelona
Manchester City
Inter
FC Barcelona
FC Bayern
Juventus
PSG
FC Bayern
Manchester Utd
Bayer 04
Spurs
FC Bayern
FC Barcelona
FC Bayern
FC Barcelona
Liverpool
PSG
Chelsea
PSG
Roma
Real Madrid
Juventus
Napoli


In [46]:
rating=pag.select("tbody > tr > td:nth-of-type(3)")
for a in rating:
    print (a.get_text()[:2])

88
88
87
87
87
87
87
87
87
87
87
87
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86


# 선수별 데이터 가져오기

In [179]:
b=get

<p>GK Reflexes <span class="pull-right"><span class="label rating r2">84</span></span></p>

In [20]:
def get_player_data(player_url):
    req=res.get(player_url["url"])
    page=BeautifulSoup(req.content,"lxml")
    stat=page.find_all("p")
    
    
    pattern="(\D)*(\d)*"
    player={}
    for skill in stat[2:-1]:
        if "Player Work Rate Medium" in skill.get_text():
            continue
        elif "Skill Moves" in skill.get_text():
            continue
        elif "Birth Date" in skill.get_text():
            continue
        elif "Kit Number" in skill.get_text():
            continue
        elif "Contract Length" in skill.get_text():
            continue
        elif "Weak Foot" in skill.get_text():
            continue
        elif "Joined Club" in skill.get_text():
            continue
    #     print(" ".join(re.search(pattern,b.get_text()).group().split(" ")[:-1]))
        player[" ".join(re.search(pattern,skill.get_text()).group().split(" ")[:-1])]=re.search(pattern,skill.get_text()).group().split(" ")[-1]
        
    player_url.update(player)
    return player_url

    
    

In [182]:
b

[{'nation': 'Real Madrid',
  'player_name': 'Cristiano Ronaldo',
  'team': 'Portugal',
  'url': 'https://www.fifaindex.com/player/20801/cristiano-ronaldo/'},
 {'nation': 'FC Barcelona',
  'player_name': 'Lionel Messi',
  'team': 'Argentina',
  'url': 'https://www.fifaindex.com/player/158023/lionel-messi/'},
 {'nation': 'PSG',
  'player_name': 'Neymar',
  'team': 'Brazil',
  'url': 'https://www.fifaindex.com/player/190871/neymar/'},
 {'nation': 'FC Barcelona',
  'player_name': 'Luis Suárez',
  'team': 'Uruguay',
  'url': 'https://www.fifaindex.com/player/176580/luis-su%C3%A1rez/'},
 {'nation': 'FC Bayern',
  'player_name': 'Manuel Neuer',
  'team': 'Germany',
  'url': 'https://www.fifaindex.com/player/167495/manuel-neuer/'},
 {'nation': 'FC Bayern',
  'player_name': 'Robert Lewandowski',
  'team': 'Poland',
  'url': 'https://www.fifaindex.com/player/188545/robert-lewandowski/'},
 {'nation': 'Manchester Utd',
  'player_name': 'De Gea',
  'team': 'Spain',
  'url': 'https://www.fifaindex.c

In [21]:
k=(get_player_data(b[0]))

In [20]:
for dic in b:
    dic.update(get_player_data(dic["url"]))
    print(dic)

{'FK Acc.': '76', 'Jumping': '95', 'Penalties': '85', 'Preferred Positions': 'LWST', 'Sprint Speed': '91', 'Ball Control': '93', 'Heading': '88', 'GK Positioning': '14', 'GK Diving': '7', 'Preferred Foot': 'Right', 'Dribbling': '91', 'Position': 'LS', 'Long Shots': '92', 'Balance': '63', 'Strength': '80', 'Att. Position': '95', 'Curve': '81', 'Vision': '85', 'Crossing': '85', 'Acceleration': '89', 'Age': '32', 'Finishing': '94', 'GK Handling': '11', 'Interceptions': '29', 'Composure': '95', 'Height': '187', 'Aggression': '63', 'Stamina': '92', 'Marking': '22', 'Short Pass': '83', 'team': 'Portugal', 'nation': 'Real Madrid', 'Reactions': '96', 'Weight': '83', 'Long Pass': '77', 'Stand Tackle': '31', 'Volleys': '88', 'player_name': 'Cristiano Ronaldo', 'GK Kicking': '15', 'Shot Power': '94', 'Player Work Rate High /': 'Low', 'url': 'https://www.fifaindex.com/player/20801/cristiano-ronaldo/', 'Agility': '89', 'Slide Tackle': '23', 'GK Reflexes': '11'}
{'FK Acc.': '90', 'Jumping': '68', 'P

In [5]:
url="https://www.fifaindex.com/player/1179/gianluigi-buffon/fifa17_173/"

req=res.get(url)
page=BeautifulSoup(req.content,"lxml")
a=page.find_all("div",class_="panel-body")


In [31]:
from lxml import html
import requests

page = requests.get('http://econpy.pythonanywhere.com/ex/001.html')
tree = html.fromstring(page.content)
#This will create a list of buyers:
buyers = tree.xpath('//div[@title="buyer-name"]/text()')
#This will create a list of prices
prices = tree.xpath('//span[@class="item-price"]/text()')


In [7]:
import re

In [168]:
pattern="(\D)*(\d)*"
player={}
for b in a[2:-1]:
    if "Player Work Rate Medium" in b.get_text():
        continue
    elif "Skill Moves" in b.get_text():
        continue
    elif "Birth Date" in b.get_text():
        continue
    elif "Kit Number" in b.get_text():
        continue
    elif "Contract Length" in b.get_text():
        continue
    elif "Weak Foot" in b.get_text():
        continue
    elif "Joined Club" in b.get_text():
        continue
#     print(" ".join(re.search(pattern,b.get_text()).group().split(" ")[:-1]))
    player[" ".join(re.search(pattern,b.get_text()).group().split(" ")[:-1])]=re.search(pattern,b.get_text()).group().split(" ")[-1]
    
print(player)
print(len(player))

{'Stand Tackle': '11', 'GK Diving': '87', 'Aggression': '38', 'Jumping': '75', 'Agility': '55', 'GK Positioning': '90', 'Weight': '94', 'Slide Tackle': '11', 'Penalties': '22', 'Preferred Positions': 'GK', 'Ball Control': '28', 'Sprint Speed': '43', 'GK Reflexes': '84', 'Marking': '10', 'Balance': '49', 'Acceleration': '49', 'Height': '191', 'Stamina': '39', 'Att. Position': '12', 'Reactions': '80', 'Strength': '69', 'Volleys': '17', 'GK Handling': '88', 'GK Kicking': '68', 'Preferred Foot': 'Right', 'Vision': '50', 'Finishing': '15', 'Long Pass': '35', 'FK Acc.': '13', 'Curve': '20', 'Dribbling': '26', 'Interceptions': '28', 'Age': '39', 'Shot Power': '39', 'Heading': '13', 'Short Pass': '37', 'Long Shots': '13', 'Position': 'GK', 'Crossing': '13', 'Composure': '80'}
40


In [63]:
import Queue

ImportError: No module named 'Queue'